In [ ]:
from google.colab import drive
drive.mount('../gdrive')

In [9]:
import pandas as pd

In [ ]:
# data에 저장된 etri-bert 모델 가져와서 압축해제
!cp "../gdrive/My Drive/data/1_bert_download_001_bert_morp_pytorch.zip" "1_bert_download_001_bert_morp_pytorch.zip"
!unzip "1_bert_download_001_bert_morp_pytorch.zip"

In [ ]:
!cp "../gdrive/My Drive/data/bert_data.zip" "bert_data.zip"
!unzip "bert_data.zip"

In [15]:
%cd ~/KorBertSum/

/opt/ml/KorBertSum


In [18]:
%ls

001_bert_morp_pytorch/          bert_config_uncased_base.json  pyrouge/
LICENSE                         bert_data/                     raw_data/
Newsdata_extractive_summ.ipynb  etri_api_scraper.py            results/
Newsdata_summarybot.ipynb       json_data/                     src/
README.md                       logs/                          summbot.png
article_to_json.py              models/                        urls/


In [11]:
!jupyter kernelspec list

[ListKernelSpecs] WARNING | Config option `kernel_spec_manager_class` not recognized by `ListKernelSpecs`.
Unable to load extension: pydevd_plugins.extensions.types.pydevd_plugin_pandas_types
Available kernels:
  my_kernel    /opt/ml/.local/share/jupyter/kernels/my_kernel
  py36         /opt/ml/.local/share/jupyter/kernels/py36
  python3.6    /opt/ml/.local/share/jupyter/kernels/python3.6
  python3      /opt/conda/share/jupyter/kernels/python3


In [13]:
%rm -r pyrouge

rm: cannot remove 'pyrouge': No such file or directory


# 1. PYROUGE (ROGUE METRIC PACKAGE)와 의존성 패키지 설치

In [24]:
# install bheinzerling's pyrouge
!git clone https://github.com/bheinzerling/pyrouge
%cd pyrouge
!python setup.py install
# install missing dependency
!apt install libxml-parser-perl
%cd pyrouge
!git clone https://github.com/andersjo/pyrouge.git rouge
!pyrouge_set_rouge_path '/opt/ml/KorBertSum/pyrouge/pyrouge/rouge/tools/ROUGE-1.5.5'
%cd /opt/ml/KorBertSum/pyrouge/pyrouge/rouge/tools/ROUGE-1.5.5/data
!mv WordNet-2.0.exc.db WordNet-2.0.exc.db.orig
!perl WordNet-2.0-Exceptions/buildExeptionDB.pl ./WordNet-2.0-Exceptions ./smart_common_words.txt ./WordNet-2.0.exc.db

Cloning into 'pyrouge'...
remote: Enumerating objects: 551, done.
remote: Total 551 (delta 0), reused 0 (delta 0), pack-reused 551
Receiving objects: 100% (551/551), 123.17 KiB | 646.00 KiB/s, done.
Resolving deltas: 100% (198/198), done.
/opt/ml/KorBertSum/pyrouge/pyrouge/rouge/tools/ROUGE-1.5.5/data/pyrouge
running install
running bdist_egg
running egg_info
creating pyrouge.egg-info
writing pyrouge.egg-info/PKG-INFO
writing dependency_links to pyrouge.egg-info/dependency_links.txt
writing top-level names to pyrouge.egg-info/top_level.txt
writing manifest file 'pyrouge.egg-info/SOURCES.txt'
reading manifest file 'pyrouge.egg-info/SOURCES.txt'
writing manifest file 'pyrouge.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/pyrouge
copying pyrouge/test.py -> build/lib/pyrouge
copying pyrouge/Rouge155.py -> build/lib/pyrouge
copying pyrouge/__init__.py -> build/lib/pyroug

pyrouge 동작 Test (마지막 출력에 ok가 나오면 성공!)

In [25]:
!python -m pyrouge.test

2023-01-16 05:01:53,093 [MainThread  ] [INFO ]  Written ROUGE configuration to /tmp/tmp7ithp0tk/rouge_conf.xml
.2023-01-16 05:01:53,329 [MainThread  ] [INFO ]  Processing files in data/SL2003_models_plain_text.
2023-01-16 05:01:53,330 [MainThread  ] [INFO ]  Processing SL.P.10.R.A.SL062003-01.html.
2023-01-16 05:01:53,330 [MainThread  ] [INFO ]  Processing SL.P.10.R.A.SL062003-21.html.
2023-01-16 05:01:53,330 [MainThread  ] [INFO ]  Processing SL.P.10.R.A.SL062003-02.html.
2023-01-16 05:01:53,330 [MainThread  ] [INFO ]  Processing SL.P.10.R.A.SL062003-17.html.
2023-01-16 05:01:53,330 [MainThread  ] [INFO ]  Processing SL.P.10.R.A.SL062003-20.html.
2023-01-16 05:01:53,330 [MainThread  ] [INFO ]  Processing SL.P.10.R.A.SL062003-05.html.
2023-01-16 05:01:53,330 [MainThread  ] [INFO ]  Processing SL.P.10.R.A.SL062003-23.html.
2023-01-16 05:01:53,331 [MainThread  ] [INFO ]  Processing SL.P.10.R.A.SL062003-07.html.
2023-01-16 05:01:53,331 [MainThread  ] [INFO ]  Processing SL.P.10.R.A.SL0620

In [26]:
# 기타 패키지 설치
!pip install pytorch_pretrained_bert
!pip install tensorboardX

# 2. GOOGLE DRIVE MOUNT

In [2]:
# Bertsum directory chdir
import os
os.chdir('/opt/ml/KorBertSum/src')

In [3]:
%ls

__pycache__/    models/  prepro/        req.txt           train.py
distributed.py  others/  preprocess.py  requirements.txt


# Train

**Arguments**

-mode train (train, validate, test) # 학습 모드 설정

-encoder classifier (classifier, transformer, RNN) # 학습 방법 설정

-dropout 0.1 # dropout rate

-bert_data_path ../bert_data/korean # bert.pt파일 저장되어있는 경로(pt파일 앞의 단어까지 경로에 쓸 것)

-model_path ../models/bert_classifier_sample # train의 경우 학습된 모델 저장

-visible_gpus 0 # -1일 경우 cpu, 0이상일 경우 cuda 사용

In [4]:
!ls

__pycache__	models	prepro	       req.txt		 train.py
distributed.py	others	preprocess.py  requirements.txt


In [7]:
!pip install -r requirements.txt

  Using cached charset_normalizer-3.0.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (195 kB)
  Using cached Cython-0.29.33-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached filelock-3.9.0-py3-none-any.whl (9.7 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached importlib_metadata-6.0.0-py3-none-any.whl (21 kB)
  Using cached kobert_transformers-0.5.1-py3-none-any.whl (12 kB)
  Using cached multiprocess-0.70.9.tar.gz (1.6 MB)
  Using cached mxnet-1.7.0.post1-py2.py3-none-manylinux2014_x86_64.whl (55.0 MB)
  Using cached numpy-1.17.2.zip (6.5 MB)
  Using cached packaging-23.0-py3-none-any.whl (42 kB)
  Using cached Pillow-9.4.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Using cached pip-22.3.1-py3-none-any.whl (2.1 MB)
  Using cached protobuf-3.20.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Using cached PyYAML

In [1]:
!python --version

Python 3.8.5


In [9]:
!python train.py -mode train -encoder classifier -dropout 0.1 -bert_data_path ../bert_data/korean -model_path ../models/bert_classifier -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 50 -save_checkpoint_steps 1000 -batch_size 1000 -decay_method noam -train_steps 1000 -accum_count 1 -log_file ../logs/bert_classifier -use_interval true -warmup_steps 8000 -bert_model ../001_bert_morp_pytorch -bert_config_path ../001_bert_morp_pytorch/bert_config.json -temp_dir .

[2023-01-16 08:35:21,802 INFO] Device ID 0
[2023-01-16 08:35:21,802 INFO] Device cuda
Traceback (most recent call last):
  File "train.py", line 367, in <module>
    train(args, device_id)
  File "train.py", line 269, in train
    torch.cuda.set_device(device_id)
  File "/opt/conda/lib/python3.8/site-packages/torch/cuda/__init__.py", line 261, in set_device
    torch._C._cuda_setDevice(device)
AttributeError: module 'torch._C' has no attribute '_cuda_setDevice'


In [ ]:
!python train.py -mode validate -bert_data_path ../bert_data/korean -model_path ../models/bert_classifier  -visible_gpus 0  -gpu_ranks 0 -batch_size 1000  -log_file ../logs/bert_classifier  -result_path ../results/korean -bert_model ../001_bert_morp_pytorch

In [ ]:
!python train.py -mode test -bert_data_path ../bert_data/korean -model_path ../models/bert_classifier  -visible_gpus 0  -gpu_ranks 0 -batch_size 1000  -log_file ../logs/bert_classifier  -result_path ../results/korean -bert_model ../001_bert_morp_pytorch -test_from ../models/bert_classifier/model_step_1000.pt

In [4]:
!ls

001_bert_morp_pytorch		bert_data	     results
LICENSE				etri_api_scraper.py  src
Newsdata_extractive_summ.ipynb	json_data	     summbot.png
Newsdata_summarybot.ipynb	logs		     temp
README.md			models		     urls
article_to_json.py		pyrouge
bert_config_uncased_base.json	raw_data


# Test

In [5]:
with open('./results/korean_step1000.gold','r') as f:
    gold = f.readlines()

In [6]:
with open('./results/korean_step1000.candidate','r') as f:
    candidate = f.readlines()

In [7]:
# 라벨 데이터
gold[3]

'인천/NNP 은/JX 산둥성/NNP 을/JKO 비롯하/VV 어/EC 산둥성/NNP 주요/NNG 도시/NNG 이/VCP ㄴ/ETM 칭다오/NNP (/SS 靑島/SH )/SS ·/SP 옌타이/NNP (/SS 煙臺/SH )/SS 시/NNG 와/JC 우호/NNG 도시/NNG 관계/NNG 이/VCP 다/EF<q>또/MAG 미래/NNG 인/NNG 적/XSN 자산/NNG 으로서/JKB 청/NNG 소년/NNG 및/MAJ 학생/NNG 들/XSN 의/JKG 교류/NNG 활성/NNG 화/XSN 필요/NNG 성/XSN 도/JX 강조/NNG 하/XSV 었/EP 다/EF<q>박남춘/NNP 시장/NNG 은/JX "/SS 두/MM 지역/NNG 의/JKG 상생/NNG 발전/NNG 을/JKO 위하/VV ㄴ/ETM 기술/NNG 협력/NNG ,/SP 상호/NNG 인증/NNG ,/SP 문화/NNG ·/SP 관광/NNG 융합/NNG 콘텐츠/NNG 개발/NNG 및/MAJ 한중/NNP FTA/SL 지방/NNG 경제/NNG 협력/NNG 시범/NNG 사업/NNG 의/JKG 성과/NNG 를/JKO 발전/NNG 시키/XSV 어/EC 두/MM 나라/NNG 간/NNB FTA/SL 선도/NNG 지역/NNG 의/JKG 역할/NNG 을/JKO 지속/NNG 적/XSN 으로/JKB 실천/NNG 하/XSV ㄹ/ETM 필요/NNG 가/JKS 있/VA 다/EF "/SS 며/EC "/SS 미래/NNG 지향/NNG 적/XSN 가치/NNG 창출/NNG 을/JKO 위하/VV 어/EC 공동/NNG 과제/NNG 를/JKO 발굴/NNG 하/XSV 고/EC 추진/NNG 하/XSV 어/EC 나가/VX 자/EF "/SS 고/JKQ 말/NNG 하/XSV 었/EP 다/EF\n'

In [8]:
# 예측 데이터
candidate[3]

'박/NNP 시장/NNG 은/JX 이날/NNG 류/NNP 서기/NNG 와/JKB 의/JKG 접견/NNG 에서/JKB 시/NNG 와/JC 산둥성/NNP 간/NNB 우호/NNG 교류/NNG 협력/NNG 관계/NNG 를/JKO 한층/MAG 격상/NNG 시키/XSV ㄹ/ETM ‘/SS (/SS 가칭/NNG )/SS 인천/NNP -/SO 산둥/NNP 포럼/NNG ’/SS 공동/NNG 추진/NNG 을/JKO 제안/NNG 하/XSV 었/EP 다/EF<q>박/NNP 시장/NNG ,/SP 류자/NNP 이/JKS 서기/NNG 와/JC 접견/NNG 공동/NNG 포럼/NNG ·/SP 청/NNG 소년/NNG 교류/NNG 제안/NNG 인천시/NNP 가/JKS 중국/NNP 산둥/NNP (/SS 山東/SH )/SS 성/NNG 과/JC 우호/NNG 관계/NNG 를/JKO 넘/VV 어/EC ‘/SS 친구/NNG 도시/NNG ’/SS 로/JKB 나아가/VV 는/ETM 첫/MM 발/NNG 을/JKO 내디디/VV 었/EP 다/EF<q>특히/MAG 인천/NNP 지역/NNG 기초/NNG 자치/NNG 단체/NNG 중/NNB 6/SN 개/NNB 군/NNG ·/SP 구/NNG 가/JKS 산둥성/NNP 11/SN 개/NNB 도시/NNG 와/JC 우호/NNG 결연/NNG 을/JKO 맺/VV 고/EC 있/VX 는/ETM 등/NNB 인천/NNP 과/JC 산둥성/NNP 은/JX 활발하/VA ㄴ/ETM 교류/NNG 가/JKS 이루/VV 어/EC 지/VV 고/EC 있/VX 다/EF\n'